In [2]:
import numpy as np
import pandas as pd
df=pd.read_csv("orders.csv",sep=',')

In [3]:
df.head(5)

,date,vendor_id,chain_id,city_id,spec,successful_orders,fail_orders
0,2019-07-02,40065,7501,23,Суши,54.0,1.0
1,2019-08-27,48058,33226,23,Шаурма,0.0,0.0
2,2019-09-25,35430,26220,25,Пицца,2.0,0.0
3,2019-09-21,56553,38601,23,Шашлыки,1.0,0.0
4,2019-09-21,43919,30984,25,Суши,6.0,0.0


In [4]:
#1
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96118 entries, 0 to 96117
Data columns (total 7 columns):
date                 96118 non-null object
vendor_id            96118 non-null int64
chain_id             96118 non-null int64
city_id              96118 non-null int64
spec                 95733 non-null object
successful_orders    96118 non-null float64
fail_orders          96118 non-null float64
dtypes: float64(2), int64(3), object(2)
memory usage: 5.1+ MB


In [5]:
#2
df.shape[0]*df.shape[1]

672826

In [9]:
#3
print(df['city_id'].unique())
df['city_id'].nunique()

[23 25 26 24]


4

In [86]:
#4.1 Wrong
tmp=df.groupby(by=['date'])['successful_orders'].sum().reset_index()
otv=tmp['successful_orders'].mean()/df['vendor_id'].nunique()
otv

4.588083472687482

In [87]:
#4.2 Right
tmp=df.groupby(by=['date'])['successful_orders'].mean().reset_index()
otv=tmp['successful_orders'].mean()
otv

8.911143419887164

In [93]:
#4.3 Right
tmp=df.groupby(by=['date','vendor_id'])['successful_orders'].sum().reset_index()
tmp=tmp.groupby(by=['date'])['successful_orders'].mean().reset_index()
tmp['successful_orders'].mean()

8.912792806580711

In [29]:
#5
df[df['spec']=='Рыба']['vendor_id'].nunique()

18

In [42]:
#6
df40065=df[df['vendor_id']==40065]
df40065=df40065.groupby(by=['date'])['successful_orders'].sum().reset_index()
df40065=df40065[df40065['successful_orders']<20]
df40065.shape[0]

45

In [85]:
#7
tmp=df.groupby(by=['vendor_id'])['successful_orders'].sum().reset_index()
tmp['successful_orders'].mean()

555.1581001951854

In [49]:
#8
tmp=df.groupby(by=['city_id'])['successful_orders'].sum().reset_index()
m=tmp['successful_orders'].min()
tmp[tmp['successful_orders']==m]

,city_id,successful_orders
1,24,114977.0


In [57]:
#9
tmp1=df.groupby(by=['spec'])['successful_orders'].sum().reset_index()
tmp2=df.groupby(by=['spec'])['fail_orders'].sum().reset_index()
tmp3=pd.merge(tmp1,tmp2,on='spec')
tmp3['ratio']=tmp3['fail_orders']/(tmp3['successful_orders']+tmp3['fail_orders'])
m=tmp3['ratio'].max()
tmp3[tmp3['ratio']==m]

,spec,successful_orders,fail_orders,ratio
1,Вегетарианское меню,5.0,24.0,0.827586


In [75]:
#10
d07=df[np.logical_and(df['date']>='2019-07-01',df['date']<='2019-07-31')]
d08=df[np.logical_and(df['date']>='2019-08-01',df['date']<='2019-08-31')]
d07=d07.groupby(by=['chain_id'])['successful_orders'].sum().reset_index()
d08=d08.groupby(by=['chain_id'])['successful_orders'].sum().reset_index()
d07=d07.rename(columns={'successful_orders':'successful_orders07'})
d08=d08.rename(columns={'successful_orders':'successful_orders08'})
d=pd.merge(d07,d08,on='chain_id',how='inner')
d['rost']=d['successful_orders08']-d['successful_orders07']
m=d['rost'].max()
d[d['rost']==m]

,chain_id,successful_orders07,successful_orders08,rost
201,29925,11167.0,12561.0,1394.0
